# Task 2 – Technical Indicators with TA-Lib / PyNance

This notebook computes basic technical indicators (MA, RSI, MACD) for one stock and visualizes them.

You can either:
- Load OHLCV data from a CSV in `data/raw/`, or
- Download it on the fly using `yfinance`.

Adjust the ticker and date range as needed.


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf

from src.config import ensure_data_dirs
from src.data_loading import load_prices_csv
from src.technical_indicators import add_moving_averages, add_rsi, add_macd

sns.set(style="whitegrid")
ensure_data_dirs()

TICKER = "AAPL"
START = "2020-01-01"
END = None  # use latest

# Option 1: download with yfinance
prices = yf.download(TICKER, start=START, end=END)
prices = prices.reset_index().rename(columns={"Date": "Date"})

# Option 2: comment the above and load from CSV in data/raw
# prices = load_prices_csv("AAPL_prices.csv", date_col="Date")

print(prices.head())


In [ ]:
# Add indicators

prices_ind = add_moving_averages(prices, close_col="Close", windows=(5, 20, 50))
prices_ind = add_rsi(prices_ind, close_col="Close", period=14)
prices_ind = add_macd(prices_ind, close_col="Close")

prices_ind.tail()


In [ ]:
# Price with moving averages

plt.figure(figsize=(14, 6))
plt.plot(prices_ind["Date"], prices_ind["Close"], label="Close", linewidth=1)
for w in (5, 20, 50):
    plt.plot(prices_ind["Date"], prices_ind[f"SMA_{w}"], label=f"SMA {w}")
plt.title(f"{TICKER} price with moving averages")
plt.xlabel("Date")
plt.ylabel("Price")
plt.legend()
plt.tight_layout()


In [ ]:
# RSI

plt.figure(figsize=(14, 3))
plt.plot(prices_ind["Date"], prices_ind[[c for c in prices_ind.columns if c.startswith("RSI_")][0]])
plt.axhline(30, color="red", linestyle="--", alpha=0.5)
plt.axhline(70, color="green", linestyle="--", alpha=0.5)
plt.title(f"{TICKER} RSI")
plt.xlabel("Date")
plt.ylabel("RSI")
plt.tight_layout()


In [ ]:
# MACD

fig, axes = plt.subplots(2, 1, figsize=(14, 6), sharex=True)

axes[0].plot(prices_ind["Date"], prices_ind["Close"], label="Close")
axes[0].set_ylabel("Price")
axes[0].set_title(f"{TICKER} price")

axes[1].plot(prices_ind["Date"], prices_ind["MACD"], label="MACD")
axes[1].plot(prices_ind["Date"], prices_ind["MACD_signal"], label="Signal")
axes[1].bar(prices_ind["Date"], prices_ind["MACD_hist"], label="Hist", alpha=0.4)
axes[1].set_ylabel("MACD")
axes[1].set_title(f"{TICKER} MACD")
axes[1].legend()

plt.tight_layout()
